In [1]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. feature_engineering
# ==========================================

def feature_engineering(df):
    
    # 수치형 변환 ('회', ' 이상' 등을 제거하고 공백 정리 후 숫자형으로 변환)
    count_cols = [
        '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수', '총 출산 횟수',
        'IVF 시술 횟수', 'DI 시술 횟수', 'IVF 임신 횟수', 'DI 임신 횟수',
        'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '이식된 배아 수', 
        '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '저장된 배아 수'
    ]
    
    for col in count_cols:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace(r'회| 이상', '', regex=True).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce')

    
    # 시술 유형 DI에 대한 결측치 처리 (0으로 채우기)
    target_cols = [
            '총 생성 배아 수', '기증자 정자와 혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
            '미세주입된 난자 수', '혼합된 난자 수', '저장된 신선 난자 수', '해동 난자 수',
            '해동된 배아 수', '미세주입 후 저장된 배아 수', '저장된 배아 수',
            '미세주입 배아 이식 수', '이식된 배아 수', '미세주입에서 생성된 배아 수',
            '수집된 신선 난자 수', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '대리모 여부',
            '기증 배아 사용 여부', '신선 배아 사용 여부', '동결 배아 사용 여부'
        ]
    existing_target_cols = [c for c in target_cols if c in df.columns]
    mask_di = df['시술 유형'] == 'DI'
    df.loc[mask_di, existing_target_cols] = df.loc[mask_di, existing_target_cols].fillna(0)

    
    # 배아 생성 이유 정제
    def clean_reason(row):
        if row['시술 유형'] == 'DI':
            return '해당없음'
    
        x = row['배아 생성 주요 이유']
        if pd.isna(x): return 'Unknown'
    
        x = str(x)
        if '시술용' in x: return '시술용'
        if '기증' in x: return '기증용'
        if '저장' in x: return '저장용'
        return 'Other'
    
    df['배아 생성 주요 이유'] = df.apply(clean_reason, axis=1)

    
    # 이진 분류 컬럼 결측 0으로 채우기
    binary_cols = ['PGS 시술 여부', 'PGD 시술 여부', '착상 전 유전 검사 사용 여부']
    df[binary_cols] = df[binary_cols].fillna(0)
    
    
    if '특정 시술 유형' in df.columns:
        df["BLASTOCYST_포함"] = df["특정 시술 유형"].str.contains("BLASTOCYST", case=False, na=False).astype(int)
        df["AH_포함"] = df["특정 시술 유형"].str.contains("AH", case=False, na=False).astype(int)
        
        def major_procedure(x):
            if pd.isna(x): return "Unknown"
            x = str(x).upper()

            # 1. ICSI (ICSI:IVF나 ICSI:ICSI 등을 모두 포함)
            if "ICSI" in x: return "ICSI"
            # 2. IVF (ICSI가 없으면서 IVF가 포함된 경우)
            if "IVF" in x: return "IVF"
            # 3. DI (기증자 인공수정)
            if "DI" in x: return "DI"
            # 4. IUI, ICI, IVI (모두 넓은 의미의 인공수정 계열)
            if any(keyword in x for keyword in ["IUI", "ICI", "IVI"]): return "IUI"
            # 5. 기타 (GIFT, FER 등 아주 적은 수의 데이터)
            if "UNKNOWN" in x: return "Unknown"
            return "Other"
        
        df["특정 시술 유형"] = df["특정 시술 유형"].apply(major_procedure)

    
    # 불임 원인 및 복잡도 계산
    female_cols = [
        '불임 원인 - 여성 요인', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', 
        '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증'
    ]
    male_cols = [
        '불임 원인 - 남성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', 
        '불임 원인 - 정자 형태', '불임 원인 - 정자 면역학적 요인'
    ]
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]

    df['남성 결함 개수'] = df[male_cols].sum(axis=1)
    df['여성 결함 개수'] = df[female_cols].sum(axis=1)
    df['불임 원인 개수'] = df[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임 원인 개수'] = df['불임 원인 개수'].apply(infertility_complexity)

    
    # 총 시술 횟수 -> 3구간으로 구간화
    def collapse_trials(x):
        if pd.isna(x): return 'Unknown'
        if x <= 0: return '0회' 
        elif 1 <= x <= 2: return '1–2회'
        elif 3 <= x <= 5: return '3–5회'
        else: return '6회 이상'

    df["총 시술 bin3"] = df["총 시술 횟수"].apply(collapse_trials)


    def div(num1, num2):
        return np.where(num2 == 0, -1, num1 / num2)

    if '시술 당시 나이' in df.columns:
        temp_age = df['시술 당시 나이'].astype(str).str.extract(r'(\d+)')[0].astype(float)
    else:
        temp_age = np.nan
    
    df['나이x이식배아'] = (temp_age + 1) * df['이식된 배아 수']
    df['연령별 배아 효율'] = div(df['이식된 배아 수'], (temp_age + 1))
    
    df['배아 생성 효율'] = div(df['총 생성 배아 수'], df['수집된 신선 난자 수'])
    df['배아 이식 비율'] = div(df['이식된 배아 수'], df['총 생성 배아 수'])
    df['배아 저장 비율'] = div(df['저장된 배아 수'], df['총 생성 배아 수'])
    
    df['이상적 배양 기간'] = df['배아 이식 경과일'].isin([3, 5]).astype(int)

    df['IVF 시술 대비 임신율'] = div(df['IVF 임신 횟수'], df['IVF 시술 횟수'])
    df['IVF 시술 대비 출산율'] = div(df['IVF 출산 횟수'], df['IVF 시술 횟수'])
    df['IVF 임신 유지력'] = div(df['IVF 출산 횟수'], df['IVF 임신 횟수'])

    df['DI 시술 대비 임신율'] = div(df['DI 임신 횟수'], df['DI 시술 횟수'])
    df['DI 시술 대비 출산율'] = div(df['DI 출산 횟수'], df['DI 시술 횟수'])
    df['DI 임신 유지력'] = div(df['DI 출산 횟수'], df['DI 임신 횟수'])

    
    # 시술 중단 단계
    def define_detailed_stage(row):
        if row['이식된 배아 수'] > 0: # 1. 이식이 실제로 이루어진 경우 (가장 성공에 근접)
            return '4_이식완료'
        elif row['저장된 배아 수'] > 0: # 2. 이식은 안 했지만, 냉동 보관된 배아가 있는 경우 (이번엔 실패/연기지만 배아는 확보됨)
            return '3_전체동결(이식보류)'
        elif row['총 생성 배아 수'] > 0: # 3. 배아는 생성되었으나 이식도 못 하고 저장도 못한 경우 (배아 발달 중지 등)
            return '2_배아생성후_소실'
        elif row['수집된 신선 난자 수'] > 0 or row['혼합된 난자 수'] > 0: # 4. 난자는 채취했으나 수정(배아 생성)에 실패한 경우
            return '1_수정실패'
        else: # 5. 난자 채취조차 실패했거나 기록이 없는 경우
            return '0_채취단계_실패_또는_정보없음'

    df['상세_진행_단계'] = df.apply(define_detailed_stage, axis=1)
    

    # 이식 취소 원인
    def explain_cancellation(row):
        if row['이식된 배아 수'] > 0:
            return 'Normal_Transfer' # 정상 이식
        if row['총 생성 배아 수'] == 0:
            return 'Fail_No_Embryos' # 배아가 없어서 못함
        if row['저장된 배아 수'] > 0:
            return 'Freeze_All' # 의도적인 전체 동결 (Freeze-all 전략 등)
        if row['착상 전 유전 진단 사용 여부'] == 1:
            return 'PGT_Fail' # 유전 검사 탈락 가능성
        return 'Unknown_Fail'

    df['이식_취소_사유'] = df.apply(explain_cancellation, axis=1)

    # 배아 생존율 및 생산 효율
    df['최종_배아_활용률'] = div(df['이식된 배아 수'] + df['저장된 배아 수'], df['수집된 신선 난자 수'])
    df['미세주입_성공률'] = div(df['미세주입에서 생성된 배아 수'], df['미세주입된 난자 수'])
    
    df.drop('ID', errors='ignore', inplace=True)
    
    return df

train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

# ==========================================
# 3. 데이터 타입 최적화
# ==========================================

def optimize_memory(df):
    for col in df.columns:
        # 숫자형 데이터 최적화
        if df[col].dtype in ['int64', 'float64']:
            if df[col].dtype == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            else:
                df[col] = pd.to_numeric(df[col], downcast='float')
        
        # 문자열(object) 데이터 -> 범주형(category) 변환
        elif df[col].dtype == 'object':
            df[col] = df[col].astype('category')
    return df

train_df = optimize_memory(train_df)
test_df = optimize_memory(test_df)

# ==========================================
# 4. 모델 학습 설정
# ==========================================

predictor = TabularPredictor(
    label=target,
    eval_metric='roc_auc',
    problem_type='binary',
    path='ag_models_out_v18',
).fit(
    train_data=train_df,
    time_limit=14400,
    presets='best_quality',
    num_bag_folds=10,
    num_bag_sets=1,
    num_stack_levels=2,
    refit_full=True,
    dynamic_stacking=False,
    save_space=True,
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       7.06 GB / 16.00 GB (44.2%)
Disk Space Avail:   119.60 GB / 460.43 GB (26.0%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=10, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 14400s
AutoGluon will save models to "/Users/admin/00 임신 예측/pregnancy-outcome-prediction-ai/Notebooks/ag_models_out_v18"
Train Data Rows:    256351
Train Data Columns

In [7]:
# --- 리더보드 출력 ---
lb = predictor.leaderboard()
display(lb.head())

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L4,0.743988,roc_auc,217.584635,12216.968628,0.030687,36.397715,4,True,52
1,WeightedEnsemble_L3,0.743985,roc_auc,145.113390,6593.854918,0.029245,15.932032,3,True,46
2,LightGBM_BAG_L3,0.743154,roc_auc,217.553948,12180.570913,1.426397,7.318608,3,True,48
3,LightGBMXT_BAG_L3,0.743042,roc_auc,217.838494,12181.584684,1.710942,8.332379,3,True,47
4,CatBoost_BAG_L3,0.742741,roc_auc,216.353190,13395.192185,0.225639,1221.939880,3,True,51


In [3]:
# --- 제출 파일 생성 ---
submission = pd.read_csv('../Data/sample_submission.csv')
submission['probability'] = final_probs.values

# 현재 시간 가져오기 (예: 0206_1031)
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"{now}_submission.csv"
submission.to_csv(file_name, index=False)

print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

학습 및 예측이 완료되었습니다. 결과가 0212_0959_submission.csv에 저장되었습니다.


In [4]:
# --- 피처 중요도 ---
# fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
# fi.to_excel('fi18.xlsx')